In [1]:
import tensorflow as tf
import numpy as np
import cv2
import cmath
import math
import os
from skimage import io

In [2]:
def caculate_gA(image):
    SAx=cv2.Sobel(image,-1,1,0)
    SAy=cv2.Sobel(image,-1,0,1)
    return (SAx*SAx+SAy*SAy)**0.5

In [3]:
def caculate_aA(image):
    SAx=cv2.Sobel(image,-1,1,0)
    SAy=cv2.Sobel(image,-1,0,1)
    t=SAy/SAx
    for i in range(t.shape[0]):
        for j in range(t.shape[1]):
            if np.isnan(t[i,j]):
                t[i,j]=10000000

    return np.arctan(t)

In [4]:
def caculate_GAF(G_A,G_F):
    m=G_A.shape[0]
    n=G_A.shape[1]
    GAF=np.zeros(shape=(m,n))

    for i in range(m):
        for j in range(n):
#             print(G_A[i,j])
            if G_A[i,j]>G_F[i,j]:
                GAF[i,j]=G_F[i,j]/G_A[i,j]
            else:
                GAF[i,j]=G_A[i,j]/G_F[i,j]
            if np.isnan(GAF[i,j]):
                GAF[i,j]=100000000
    return GAF

In [5]:
def caculate_AAF(a_A,a_F):
    return abs(abs(a_A-a_F)-math.pi/2)/(math.pi/2)

In [6]:
def caculate_QgAF(GAF):
    return 0.9994/(1+np.exp(15*(GAF-0.5)))

In [7]:
def caculate_QaAF(AAF):
    return 0.9879/(1+np.exp(22*(AAF-0.8)))

In [8]:
def caculate_QAF(QgAF,QaAF):
    return QgAF*QaAF

In [9]:
def caculate_wx(Gx):
    return Gx**1.5


In [10]:
def caculate_Q(QAF,QBF,wA,wB):
    m=QAF.shape[0]
    n=QAF.shape[1]
    sum1=0
    for i in range(m):
        for j in range(n):
            sum1=sum1+QAF[i,j]*wA[i,j]+QBF[i,j]*wB[i,j]
    sum2=0
    for i in range(m):
        for j in range(n):
            sum2=sum2+wA[i,j]+wB[i,j]
    print("sum1",sum1,"sum2",sum2)
    return (sum1/sum2*10)

In [11]:
oimg_A = io.imread("data/a.png")
oimg_B = io.imread("data/b.png")
oimg_F= io.imread("data/f.png")
m=oimg_A.shape[0]
n=oimg_A.shape[1]
img_A=np.zeros(shape=(m,n))
img_B=np.zeros(shape=(m,n))
img_F=np.zeros(shape=(m,n))
img_A=oimg_A[:,:,0]+oimg_A[:,:,1]+oimg_A[:,:,2]
img_B=oimg_B[:,:,0]+oimg_B[:,:,1]+oimg_B[:,:,2]
img_F=oimg_F[:,:,0]+oimg_F[:,:,1]+oimg_F[:,:,2]

In [12]:
#计F
F_gA=caculate_gA(img_F)
F_aA=caculate_aA(img_F)

E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [13]:
#计算A
A_gA=caculate_gA(img_A)
A_aA=caculate_aA(img_A)

A_GAF=caculate_GAF(A_gA,F_gA)

A_AAF=caculate_AAF(A_aA,F_aA)
A_QgAF=caculate_QgAF(A_GAF)
A_aGAF=caculate_QaAF(A_AAF)

A_QAF=caculate_QAF(A_QgAF,A_aGAF)

WA=caculate_wx(A_gA)


E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [14]:
#计算B
B_gA=caculate_gA(img_B)
B_aA=caculate_aA(img_B)
B_GAF=caculate_GAF(B_gA,B_gA)
B_AAF=caculate_AAF(B_aA,F_aA)
B_QgAF=caculate_QgAF(B_GAF)
B_aGAF=caculate_QaAF(B_AAF)
B_QAF=caculate_QAF(B_QgAF,B_aGAF)
WB=caculate_wx(B_gA)

E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
E:\ANACONDA\envs\geospatial\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [15]:
t=caculate_Q(A_QAF,B_QAF,WA,WB)
print(t)

sum1 886026.8393959682 sum2 12021739.901706133
0.07370204701153306
